In [1]:
!pip install -q openai==1.107.0

In [2]:
import os

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
# os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"


from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## Math Tutor

In [3]:
from openai import OpenAI

client = OpenAI()

# Create a conversation for persistent context (optional)
conversation = client.conversations.create()

# Single API call with built-in code interpreter
response = client.responses.create(
    model="gpt-5",
    reasoning={'effort':'low'},
    instructions="You are a personal math tutor. When asked a math question, write and run code using the python tool to answer the question.",
    tools=[
        {
            "type": "code_interpreter",
            "container": {"type": "auto"}
        }
    ],
    input= "I need to solve the equation `3x + 11 = 14`. Can you help me?",
    conversation=conversation.id  # Optional: for persistent context
)

print(response.output_text)


Solve 3x + 11 = 14:
- Subtract 11 from both sides: 3x = 3
- Divide by 3: x = 1

Answer: x = 1


## Tech Support

In [4]:
import requests
import time
from openai import OpenAI

client = OpenAI()

# Download PDF
url = "https://personales.unican.es/corcuerp/linux/resources/LinuxCommandLineCheatSheet_1.pdf"
response = requests.get(url)
with open("LinuxCommandLineCheatSheet_1.pdf", "wb") as f:
    f.write(response.content)

# Upload file
with open("LinuxCommandLineCheatSheet_1.pdf", "rb") as file:
    uploaded_file = client.files.create(file=file, purpose="assistants")

In [5]:
# Create vector store
vector_store = client.vector_stores.create(name="Tech Support")

# Add file to vector store
client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=uploaded_file.id
)

# Wait for processing
while True:
    file_status = client.vector_stores.files.retrieve(
        vector_store_id=vector_store.id,
        file_id=uploaded_file.id
    )
    if file_status.status == "completed":
        break
    time.sleep(2)

In [6]:
# Use GPT-5 with file search
response = client.responses.create(
    model="gpt-5",
    reasoning={'effort':'low'},
    instructions="You are a tech support chatbot. Use the product manual to respond accurately to customer inquiries.",
    input="What does the 'ls' command do?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }]
)

print(response.output_text)

ls lists the files and directories in your current directory. A common usage is ls -al, which shows all files (including hidden ones) in a detailed “long” format .


In [7]:
print(response.tools)

[FileSearchTool(type='file_search', vector_store_ids=['vs_68d0dce6e1908191a7da8005ad2884aa'], filters=None, max_num_results=20, ranking_options=RankingOptions(ranker='auto', score_threshold=0.0))]


In [8]:
print(response.output[3].content[0].annotations)

[AnnotationFileCitation(file_id='file-A95fCbfWd74jwpu59jzVox', filename='LinuxCommandLineCheatSheet_1.pdf', index=162, type='file_citation')]


In [10]:
import time
from openai import OpenAI

client = OpenAI()

files = client.files.list(purpose="assistants")
# The response contains a list of file metadata objects, including id and filename
for f in files.data:
    if f.filename == "LinuxCommandLineCheatSheet_1.pdf":
        file_id = f.id
        break

# This permanently removes the file from your assistant files
client.files.delete(file_id)

# Poll the list endpoint until the file no longer appears
while True:
    current_files = client.files.list(purpose="assistants").data
    if all(f.id != file_id for f in current_files):
        print(f"File {file_id} successfully deleted.")
        break
    time.sleep(1)


File file-A95fCbfWd74jwpu59jzVox successfully deleted.


In [9]:
# # Clean up
# os.remove("LinuxCommandLineCheatSheet_1.pdf")